In [0]:
!pip install mlflow transformers torch scikit-learn nltk
%restart_python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 108.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 117.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 122.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.2/106.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [0]:
import nltk
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report

# Unified Fallacy Pipeline

In [0]:
import nltk
nltk.download('punkt')

def load_fallacy_data(fallacy_name: str):
    # Query using Spark SQL
    examples_df = spark.sql(f"""
        SELECT text 
        FROM logical_fallacy_data.{fallacy_name}
        WHERE label = true
    """)

    non_examples_df = spark.sql(f"""
        SELECT text 
        FROM logical_fallacy_data.{fallacy_name}
        WHERE label = false
    """)

    # Convert to Pandas (safe in Databricks serverless)
    examples = examples_df.toPandas()['text'].tolist()
    non_examples = non_examples_df.toPandas()['text'].tolist()

    print(f"[{fallacy_name}] examples: {len(examples)}")
    print(f"[{fallacy_name}] non-examples: {len(non_examples)}")

    return examples, non_examples

fallacies = [
    "red_herring",
    "straw_man",
    "slippery_slope",
    "attacking",
    "ad_hominem",
    "hasty_generalization",
    "ignorance",
    "hypocrisy",
    "stacking_deck"
]

fallacy_data = {}

for name in fallacies:
    examples, non_examples = load_fallacy_data(name)
    fallacy_data[name] = {
        "examples": examples,
        "non_examples": non_examples
    }


[nltk_data] Downloading package punkt to
[nltk_data]     /home/spark-b1faa9a0-ee82-4d76-938f-b6/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[red_herring] examples: 36
[red_herring] non-examples: 38
[straw_man] examples: 1
[straw_man] non-examples: 7
[slippery_slope] examples: 1
[slippery_slope] non-examples: 7
[attacking] examples: 0
[attacking] non-examples: 0
[ad_hominem] examples: 28
[ad_hominem] non-examples: 60
[hasty_generalization] examples: 0
[hasty_generalization] non-examples: 0
[ignorance] examples: 42
[ignorance] non-examples: 37
[hypocrisy] examples: 22
[hypocrisy] non-examples: 43
[stacking_deck] examples: 16
[stacking_deck] non-examples: 15


In [0]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os

def train_fallacy_model(fallacy_name, examples, non_examples, output_dir="models", epochs=3, batch_size=8, lr=2e-5):
    if not examples or not non_examples:
        print(f"⚠️ Skipping {fallacy_name}: empty examples or non-examples.")
        return
    # Prepare training data
    texts = examples + non_examples
    labels = [1] * len(examples) + [0] * len(non_examples)

    # Load tokenizer and encode
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    encodings = tokenizer(texts, truncation=True, padding=True, return_tensors='pt')
    inputs = encodings['input_ids']
    masks = encodings['attention_mask']
    labels_tensor = torch.tensor(labels)

    # Build dataset and split
    dataset = TensorDataset(inputs, masks, labels_tensor)
    train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size)

    # Load model
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    # Train loop
    model.train()
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        for batch in train_loader:
            b_input_ids, b_input_mask, b_labels = batch
            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

    # Evaluate
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in test_loader:
            b_input_ids, b_input_mask, b_labels = batch
            outputs = model(b_input_ids, attention_mask=b_input_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            predictions.extend(preds.tolist())
            true_labels.extend(b_labels.tolist())

    print(f"\n[Evaluation for {fallacy_name}]")
    print(classification_report(true_labels, predictions))

    # Save model/tokenizer
    model_dir = os.path.join(output_dir, fallacy_name)
    os.makedirs(model_dir, exist_ok=True)
    model.save_pretrained(model_dir)
    tokenizer.save_pretrained(model_dir)
    print(f"✅ Model saved to {model_dir}")


In [0]:
# Safe temporary local storage
local_model_dir = "/local_disk0/tmp/fallacy_models"
os.makedirs(local_model_dir, exist_ok=True)

for fallacy_name in fallacies:
    examples, non_examples = load_fallacy_data(fallacy_name)
    train_fallacy_model(fallacy_name, examples, non_examples)
    if not examples or not non_examples:
        print(f"❌ No data for: {fallacy_name}")

[red_herring] examples: 36
[red_herring] non-examples: 38


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Epoch 1/3
Epoch 2/3
Epoch 3/3

[Evaluation for ignorance]
              precision    recall  f1-score   support

           0       0.71      1.00      0.83         5
           1       1.00      0.82      0.90        11

    accuracy                           0.88        16
   macro avg       0.86      0.91      0.87        16
weighted avg       0.91      0.88      0.88        16

✅ Model saved to models/ignorance
[hypocrisy] examples: 22
[hypocrisy] non-examples: 43


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
Epoch 2/3
Epoch 3/3

[Evaluation for hypocrisy]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         5

    accuracy                           1.00        13
   macro avg       1.00      1.00      1.00        13
weighted avg       1.00      1.00      1.00        13

✅ Model saved to models/hypocrisy
[stacking_deck] examples: 16
[stacking_deck] non-examples: 15


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
Epoch 2/3
Epoch 3/3

[Evaluation for stacking_deck]
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         4
           1       0.60      1.00      0.75         3

    accuracy                           0.71         7
   macro avg       0.80      0.75      0.71         7
weighted avg       0.83      0.71      0.70         7

✅ Model saved to models/stacking_deck


# DO NOT TOUCH BELOW UNRELATED

In [0]:
import os
import shutil

# Safe temporary local storage
local_model_dir = "/local_disk0/tmp/fallacy_models"
os.makedirs(local_model_dir, exist_ok=True)

# Copy only trained models into safe path
for fallacy in ["straw_man", "red_herring", "ad_hominem"]:  # Add only trained models
    src = f"models/{fallacy}"
    dst = f"{local_model_dir}/{fallacy}"
    if os.path.exists(src):
        shutil.copytree(src, dst, dirs_exist_ok=True)

In [0]:
import os
import shutil

# Target directory to simulate a proper Python module
clean_code_dir = "/local_disk0/tmp/fallacy_ensemble_module"
os.makedirs(clean_code_dir, exist_ok=True)

# Path to the final .py file
final_py_path = os.path.join(clean_code_dir, "fallacy_ensemble_model.py")

# Write the class definition cleanly
with open(final_py_path, "w", encoding="utf-8") as f:
    f.write("""
import os
import torch
import mlflow.pyfunc
from transformers import BertTokenizer, BertForSequenceClassification

class FallacyEnsembleModel(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        self.fallacy_types = [
            "straw_man", "red_herring", "ad_hominem",
            "hasty_generalization", "appeal_to_ignorance",
            "hypocrisy", "stacking_deck"
        ]
        self.model_dir = context.artifacts["model_dir"]
        self.fallacy_models = {}

        for fallacy in self.fallacy_types:
            model_path = os.path.join(self.model_dir, fallacy)
            if not os.path.exists(model_path):
                print(f"Skipping {fallacy}: model not found at {model_path}")
                continue
            try:
                model = BertForSequenceClassification.from_pretrained(model_path)
                tokenizer = BertTokenizer.from_pretrained(model_path)
                model.eval()
                self.fallacy_models[fallacy] = (tokenizer, model)
            except Exception as e:
                print(f"Error loading {fallacy}: {e}")

    def predict(self, context, model_input):
        texts = model_input["text"].tolist()
        results = []
        for text in texts:
            fallacy_scores = []
            for fallacy, (tokenizer, model) in self.fallacy_models.items():
                inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
                with torch.no_grad():
                    logits = model(**inputs).logits
                    probs = torch.softmax(logits, dim=1)
                    fallacy_prob = probs[0][1].item()
                fallacy_scores.append((fallacy, fallacy_prob))
            fallacy_scores.sort(key=lambda x: x[1], reverse=True)
            results.append(fallacy_scores)
        return results
""")

import mlflow.pyfunc
import sys
sys.path.append("/local_disk0/tmp/fallacy_ensemble_module")
from fallacy_ensemble_model import FallacyEnsembleModel

save_path = "/local_disk0/tmp/fallacy_ensemble_model_v1"

mlflow.pyfunc.save_model(
    path=save_path,
    loader_module="fallacy_ensemble_model",          # file name WITHOUT .py
    code_paths=[clean_code_dir],                     # the folder that contains the file
    python_model="FallacyEnsembleModel",             # the class name
    artifacts={"model_dir": "/local_disk0/tmp/fallacy_models"}
)



---------------------------------------------------------------------------
MlflowException                           Traceback (most recent call last)
File <command-7930981280555412>, line 66
     62 from fallacy_ensemble_model import FallacyEnsembleModel
     64 save_path = "/local_disk0/tmp/fallacy_ensemble_model_v1"
---> 66 mlflow.pyfunc.save_model(
     67     path=save_path,
     68     loader_module="fallacy_ensemble_model",          # file name WITHOUT .py
     69     code_paths=[clean_code_dir],                     # the folder that contains the file
     70     python_model="FallacyEnsembleModel",             # the class name
     71     artifacts={"model_dir": "/local_disk0/tmp/fallacy_models"}
     72 )

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-b1faa9a0-ee82-4d76-938f-b69d2f75890f/lib/python3.10/site-packages/mlflow/tracing/provider.py:422, in trace_disabled.<locals>.wrapper(*args, **kwargs)
    420 disable()
    421 try:
--> 422     is_func_called, result = True, f(

In [0]:
import os
import shutil
import mlflow.pyfunc

# Step 1: Create a clean code folder and copy the .py file into it
clean_code_dir = "/local_disk0/tmp/mlflow_fallacy_code"
os.makedirs(clean_code_dir, exist_ok=True)

shutil.copy("/local_disk0/tmp/fallacy_ensemble_model.py", f"{clean_code_dir}/fallacy_ensemble_model.py")

save_path = "/local_disk0/tmp/fallacy_ensemble_model_v1"
shutil.rmtree(save_path, ignore_errors=True)

mlflow.pyfunc.save_model(
    path=save_path,
    loader_module="fallacy_ensemble_model",      # Matches filename: fallacy_ensemble_model.py
    code_paths=[clean_code_dir],                 # Folder that contains the .py file
    python_model="FallacyEnsembleModel",         # Class name in the .py file
    artifacts={"model_dir": "/local_disk0/tmp/fallacy_models"}
)



---------------------------------------------------------------------------
MlflowException                           Traceback (most recent call last)
File <command-8027030044030855>, line 14
     11 save_path = "/local_disk0/tmp/fallacy_ensemble_model_v1"
     12 shutil.rmtree(save_path, ignore_errors=True)
---> 14 mlflow.pyfunc.save_model(
     15     path=save_path,
     16     loader_module="fallacy_ensemble_model",      # Matches filename: fallacy_ensemble_model.py
     17     code_paths=[clean_code_dir],                 # Folder that contains the .py file
     18     python_model="FallacyEnsembleModel",         # Class name in the .py file
     19     artifacts={"model_dir": "/local_disk0/tmp/fallacy_models"}
     20 )

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-b1faa9a0-ee82-4d76-938f-b69d2f75890f/lib/python3.10/site-packages/mlflow/tracing/provider.py:422, in trace_disabled.<locals>.wrapper(*args, **kwargs)
    420 disable()
    421 try:
--> 422     is_func_called, result

In [0]:
# Register it in Unity Catalog or default registry
registered_model_name = "fallacy_ensemble_uc"
mlflow.register_model(f"runs:/{model_artifact_path}", registered_model_name)

# Example Usage

In [0]:
model = mlflow.pyfunc.load_model(save_path)

import pandas as pd
df = pd.DataFrame({"text": ["You're just a student."]})
predictions = model.predict(df)
print(predictions)
